# Baby Shark Activity
Detects activity that could be related to Baby Shark malware

## Rule Content
```
- title: Baby Shark Activity
  id: 2b30fa36-3a18-402f-a22d-bf4ce2189f35
  status: experimental
  description: Detects activity that could be related to Baby Shark malware
  references:
  - https://unit42.paloaltonetworks.com/new-babyshark-malware-targets-u-s-national-security-think-tanks/
  tags:
  - attack.execution
  - attack.t1059
  - attack.t1086
  - attack.discovery
  - attack.t1012
  - attack.defense_evasion
  - attack.t1170
  logsource:
    category: process_creation
    product: windows
    service: null
  author: Florian Roth
  date: 2019/02/24
  detection:
    selection:
      CommandLine:
      - reg query "HKEY_CURRENT_USER\Software\Microsoft\Terminal Server Client\Default"
      - powershell.exe mshta.exe http*
      - cmd.exe /c taskkill /im cmd.exe
    condition: selection
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(reg\ query\ \"HKEY_CURRENT_USER\\Software\\Microsoft\\Terminal\ Server\ Client\\Default\" OR powershell.exe\ mshta.exe\ http* OR cmd.exe\ \/c\ taskkill\ \/im\ cmd.exe)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()